### Writing data to SQL database

Initially, it was considered to use SQLite in order to store data within a local file, however, to achieve up-to-date state of the database for both team members, it was decided to move the database to MariaDB on Amazon AWS. 

In [1]:
# Using SQLite within Jupyter notebooks - option 1

# %%capture
# %load_ext sql
# %sql sqlite:///vivino.db
# %sql SELECT * FROM wine

In [2]:
# Using SQLite within Jupyter notebooks - option 2

import sqlite3
conn = sqlite3.connect('vivino.db')
test_query = 'SELECT * FROM wine LIMIT 5'
print(conn.execute(test_query).fetchall())
conn.close()

[(1105374, 'Alandra Tinto', 'alandra-tinto', 1, 0, 0, 1395, 3155, 3.1399224, None, 3.7395344, 1.8591489, 2.9908836, 540, 184, 2, 'Normal', 18112, 3.2, 128391, 51, 1), (1706071, 'Alentejano Monte das Ânforas Tinto', 'alentejano-monte-das-anforas-tinto', 1, 0, 0, 1394, 4107, 3.0573564, None, 3.7483184, 1.799455, 2.8970287, 171, 38, 2, 'Normal', 5176, 3.4, 30372, 38, 1), (4269600, 'Vinea Tinto', 'vinea-tinto', 1, 0, 0, 1394, 10992, 3.0659487, None, 3.2864575, 2.008701, 2.8928797, 435, 31, 2, 'Normal', 5827, 3.3, 37482, 39, 1), (1200770, 'Lisboa Tinto', 'lisboa-tinto', 1, 0, 0, 834, 26708, 3.042016, None, 3.4385235, 1.8823165, 2.7859948, 69, 59, 210, 'Normal', 2153, 3.4, 11457, 34, 1), (4269602, 'Vinea Branco', 'vinea-branco', 2, 0, 0, 1394, 10992, 2.8494768, None, 2.7141547, 1.6764449, None, 68, 9, 211, 'Normal', 1186, 3.3, 6375, 25, 1)]


In [3]:
import mariadb

In [4]:
import sys
sys.path.append('..')
import settings

In [7]:
def connect_to_vivino_db():
    """
    connect to vivino db and return a connection instance
    """
    try:
        conn =  mariadb.connect(
                user="admin",
                password=settings.db_pass,
                host=settings.db_url,
                port=3306,
                database="vivino")
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)
    return conn

In [8]:
conn = connect_to_vivino_db()

Database schema was created separately based on the data structure, and looks as follows: 

![](vivino-schema.png)

Insertion to the database should be done in the following order (in order to avoid problems with foreign keys):
* wine type
* winery
* country
* region
* style
* food
* facts
* style_food
* grape
* style_grape
* grape_country
* wine
* vintage
* toplist
* vintage_toplist
* keyword
* wine_keyword
* wine_flavor_group

Once the database schema and the name of the columns is fixed, we can run functions that insert data to each table by extracting information from JSON file. 

Since we need only specific data from JSON, we need to specify the path to such data for each column in each table. For convenience, such path will be formed using a forward slash (for example, `vintage/wine/region/name`). 

The function extracts data found at a given path inside a given record (making one step of the path at a time), and returns the resulting value. If any of the path steps is missing for a given record, it returns the value at the latest step available. 

In order to generate the path, here is the full list of columns in the dataframe which mimics the structure of JSON:

['vintage_grapes', 'vintage_has_valid_ratings', 'vintage_id',
       'vintage_image', 'vintage_name', 'vintage_seo_name',
       'vintage_statistics', 'vintage_wine', 'vintage_year',
       'vintage_top_list_rankings', 'status', 'ratings_count',
       'ratings_average', 'labels_count', 'id', 'name', 'seo_name', 'type_id',
       'vintage_type', 'is_natural', 'has_valid_ratings', 'region.id',
       'region.name', 'region.name_en', 'region.seo_name',
       'region.country.code', 'region.country.name',
       'region.country.native_name', 'region.country.seo_name',
       'region.country.currency.code', 'region.country.currency.name',
       'region.country.currency.prefix', 'region.country.currency.suffix',
       'region.country.regions_count', 'region.country.users_count',
       'region.country.wines_count', 'region.country.wineries_count',
       'region.country.most_used_grapes', 'region.background_image.location',
       'region.background_image.variations.large',
       'region.background_image.variations.medium', 'winery.id', 'winery.name',
       'winery.seo_name', 'winery.status', 'taste.structure.acidity',
       'taste.structure.fizziness', 'taste.structure.intensity',
       'taste.structure.sweetness', 'taste.structure.tannin', 
       'taste.structure.user_structure_count',
       'taste.structure.calculated_structure_count', 'taste.flavor',
       'statistics.status', 'statistics.ratings_count',
       'statistics.ratings_average', 'statistics.labels_count',
       'statistics.vintages_count', 'style.id', 'style.seo_name',
       'style.regional_name', 'style.varietal_name', 'style.name',
       'style.image', 'style.background_image.location',
       'style.background_image.variations.small', 'style.description',
       'style.blurb', 'style.interesting_facts', 'style.body',
       'style.body_description', 'style.acidity', 'style.acidity_description',
       'style.country.code', 'style.country.name', 'style.country.native_name',
       'style.country.seo_name', 'style.country.currency.code',
       'style.country.currency.name', 'style.country.currency.prefix',
       'style.country.currency.suffix', 'style.country.regions_count',
       'style.country.users_count', 'style.country.wines_count',
       'style.country.wineries_count', 'style.country.most_used_grapes',
       'style.wine_type_id', 'style.food', 'style.grapes', 'style.region',
       'style.region.id', 'style.region.name', 'style.region.name_en',
       'style.region.seo_name', 'style.region.country.code',
       'style.region.country.name', 'style.region.country.native_name',
       'style.region.country.seo_name', 'style.region.country.currency.code',
       'style.region.country.currency.name',
       'style.region.country.currency.prefix',
       'style.region.country.currency.suffix',
       'style.region.country.regions_count',
       'style.region.country.users_count', 'style.region.country.wines_count',
       'style.region.country.wineries_count',
       'style.region.country.most_used_grapes',
       'style.region.background_image.location',
       'style.region.background_image.variations.large',
       'style.region.background_image.variations.medium', 'region', 'winery',
       'style', 'taste.structure', 'region.background_image',
       'style.background_image', 'style.region.background_image']


In [10]:
def get_value(match_entry, path0):
    """
    Function that returns a value found at a given path inside a given JSON record 
    """
    path = path0.split('/')
    current_el = match_entry
    for p in path:
        if current_el is None:
            break
        current_el = current_el.get(p)
    return current_el

In [12]:
# load data from a backup file

import pickle

with open(f"backup_data/full_match_list", 'rb') as f:
    recovered_data = pickle.load(f)
    
def remove_wine_duplicates(json_data):
    distinct_dict = {entry['vintage']['id']: entry for entry in json_data}
    recovered_data_distinct = distinct_dict.values()
    return list(recovered_data_distinct)

recovered_data_distinct = remove_wine_duplicates(recovered_data)
len(recovered_data_distinct)

55819

In [26]:
# insert data to sql
import time

def extract_json_to_sql(matches_list, table_name, paths, pk_sql, conn, first_entry=True):
    """
    Function that accepts the folowing arguments:
    * matches list: JSON list with data, 
    * table_name: name of SQL table to include data,  
    * paths: the paths leading to data in JSON corresponding to each column in SQL table,
    * pk_sql: the names of primary key columns in SQL (to check for uniqueness condition),
    * conn: active connection to a database
    * first_entry: boolean indicating whether it's the first time data is written to the table
    
    Function inserts data and (for the first entry) checks whether the resulting number of unique records in SQL
    matches the number of unique records in JSON.
    
    """
    cur = conn.cursor()
    
    # check if primary key(s) already exist in a db. 
    # Primary key column names are inserted to the string using format method, and '?' are used as placeholders for the actual values. 
    conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql) 

    timepoint_1 = time.time()
    
    # returns JSON paths to primary keys assuming that primary key(s) always come first in columns of each SQL table 
    pk_paths = [paths[i] for i in range(len(pk_sql))]         

    for entry in matches_list:
        # for each primary key check whether value in JSON exists (if any of the primary keys is missing, move to the next record)
        if any([get_value(entry, path) is None for path in pk_paths]):
            continue
        else:
            # insert all values extracted from JSON using '?' as placeholders, provided current primary keys are not yet in the db
            query = f"""
              INSERT INTO {table_name} SELECT {', '.join('?' * len(paths))}     
              WHERE NOT EXISTS (SELECT * FROM {table_name} {conditional_statement})
            """
            values_entry = [get_value(entry, path) for path in paths]
            # filling in all SQL placeholders (a value for each column, and then a value for each primary key column (assuming they come first))
            args = tuple(values_entry + values_entry[:len(pk_sql)]) 
            cur.execute(query, args)
        
    timepoint_2 = time.time()
    print('Insertion complete and took {} s.'.format(timepoint_2 - timepoint_1))

    if first_entry: 
        unique_keys = set()
        for entry in matches_list:
            unique_key = []
            for path in pk_paths:
                unique_key.append(get_value(entry, path))
            if None in unique_key:
                # do nothing
                pass
            else:
                unique_keys.add(tuple(unique_key))
        unique_pk_original = len(unique_keys)
    
#         print(unique_pk_original)
        if unique_pk_original == cur.execute('SELECT COUNT(*) FROM {}'.format(table_name)).fetchall()[0][0]:
            print('Number of unique records is accurate')
        else:
            print('Something went wrong')

In [14]:
def clean_sql_table(conn, table_name):
    """
    delete all records from a given table in a given database
    """
    cur = conn.cursor()
    cur.execute(f'DELETE FROM {table_name}')

In [15]:
def count_unique_records_sql(conn, table_name):
    """
    checks the number of unique records in a given table
    """
    cur = conn.cursor()
    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
    print(cur.fetchall())

Before writing into MariaDB we need to make sure that the data is cleaned.

In [16]:
tables = ['wine_flavor_group', 'wine_keyword', 'keyword', 'vintage_toplist', 'toplist', 'vintage', 'wine', 'grape_country', 'style_grape', \
          'style_food', 'grape', 'food', 'facts', 'style', 'region', 'country', 'price', 'winery', 'type']

conn = connect_to_vivino_db()
for table in tables: 
    clean_sql_table(conn, table)
    print(f"{table} cleaned")

wine_flavor_group cleaned
wine_keyword cleaned
keyword cleaned
vintage_toplist cleaned
toplist cleaned
vintage cleaned
wine cleaned
grape_country cleaned
style_grape cleaned
style_food cleaned
grape cleaned
food cleaned
facts cleaned
style cleaned
region cleaned
country cleaned
price cleaned
winery cleaned
type cleaned


#### Insert wine types

In [17]:
def insert_to_wine_types(conn, table_name='type'):
    """
    manually inserts all wine types to SQL table
    """
    cur = conn.cursor()
    cur.execute(f"INSERT INTO {table_name} VALUES (1, 'Red'), (2, 'White'), (3, 'Sparkling'), (4, 'Rose'), (7, 'Dessert'), (24, 'Fortified');")

In [20]:
insert_to_wine_types(conn)
cur.execute("SELECT * FROM type")
print(cur.fetchall())

[(1, 'Red'), (2, 'White'), (3, 'Sparkling'), (4, 'Rose'), (7, 'Dessert'), (24, 'Fortified')]


#### Insert wineries

In [27]:
#insert wineries

def insert_to_wineries(conn, matches):
    """
    inserts data to correct fields in the wine table
    """
    table = 'winery'
    main = 'vintage/wine/winery/'
    paths = [main + 'id', main + 'name', main + 'seo_name', main + 'status']
    pk_sql = ['id']
    extract_json_to_sql(matches, table, paths, pk_sql, conn, first_entry=True)

In [31]:
conn.cursor().execute("ALTER DATABASE vivino CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;")

In [33]:
conn.cursor().execute("ALTER TABLE winery CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

In [36]:
conn = connect_to_vivino_db()
clean_sql_table(conn, 'winery')

In [40]:
insert_to_wineries(conn, recovered_data_distinct)

InterfaceError: Lost connection to MySQL server during query

In [39]:
recovered_data_distinct[709]['vintage']['wine']['winery']

{'id': 4067, 'name': 'McGuigan', 'seo_name': 'mcguigan', 'status': 0}

In [35]:
conn.close()